# Model Training

In this exercise we will perform feature engineering on our training set then train a model, deploy it and write a [model card](https://doi.org/10.1145/3287560.3287596) to provide reporting on the model.

## Task 0 - Setup

These are just some preliminary steps for getting started with this section.

### 🔄 Task

- Load any environment variables located in `.env`
- Get the username for content deployed to Posit Connect

### 🧑‍💻 Code

In [ ]:
from pathlib import Path

if Path(".env").exists():
    from dotenv import load_dotenv

    load_dotenv()

In [ ]:
from posit.connect import Client

with Client() as client:
    username = client.me.username

print(f"Connect username is: '{username}'")

## Task 1 - Reading the data

First we will read in the data prepared in [02-data-exploration-and-validation](../02-data-exploration-and-validation/notebook.ipynb) and take a quick look at it prepare for modeling.

### 🔄 Task

- Read in and glimpse the vessel history data
- Read in and glimpse the vessel verbose data
- Read in and glimpse the weather data

### 🧑‍💻 Code

In [ ]:
import os

db_uri = os.environ["DATABASE_URI_PYTHON"]

In [ ]:
import polars as pl

vessel_history = pl.read_database_uri(
    query=f"SELECT * FROM {username}_vessel_history_clean;", uri=db_uri, engine="adbc"
)

vessel_history.head(3)

In [ ]:
vessel_verbose = pl.read_database_uri(
    query=f"SELECT * FROM {username}_vessel_verbose_clean;", uri=db_uri, engine="adbc"
)

vessel_verbose.head(3)

In [ ]:
weather = pl.read_database_uri(
    query=f"SELECT * FROM {username}_terminal_weather_clean;", uri=db_uri, engine="adbc"
)

weather.head(3)

## Task 2 - Feature Engineering

### 🔄 Task

- Join the `vessel_history`, `vessel_verbose` and `weather` data into a form useful for modeling
- Transform the columns in new ones we can use for modeling

### 🧑‍💻 Code

First we compute a departure delay based on the difference between the actual and scheduled departure times and derive new features representing the day of the week and hour of departure.

In [ ]:
trips_combined = vessel_history.with_columns(
    (pl.col("ActualDepart") - pl.col("ScheduledDepart"))
    .dt.total_seconds()
    .alias("Delay"),
    pl.col("Date").dt.weekday().alias("Weekday"),
    pl.col("Date").dt.hour().alias("Hour"),
).drop("EstArrival")

trips_combined.head(3)

Taking a look at `Delay` shows it is very skewed with a lot of values near zero and some even less than zero.

In [ ]:
trips_combined.plot.hist("Delay", bin_range=(-1800, 7200), bins=30)

Since we're not interested in negative values here, we'll clip the values below zero. We can then log them, turning into it a nicer, more "normal" distribution. 

In [ ]:
trips_combined = trips_combined.select(
    pl.exclude("Delay"),
    pl.col("Delay")
    .map_elements(lambda x: max(x, 1), return_dtype=pl.Float64)
    .log()
    .alias("LogDelay"),
)

trips_combined.plot.hist("LogDelay")

Now we prepare the data describing the individual vessels by only keeping the columns we're interested in and extracting the year from `YearBuilt` and `YearRebuilt`.

In [ ]:
vessel_info = vessel_verbose.select(
    pl.col("VesselName"),
    pl.col("ClassName"),
    # we can also select multiple columns in one `pl.col(...)`
    pl.col(
        "SpeedInKnots",
        "EngineCount",
        "Horsepower",
        "MaxPassengerCount",
        "PassengerOnly",
        "FastFerry",
        "PropulsionInfo",
    ),
    pl.col("YearBuilt", "YearRebuilt").dt.year(),
)

vessel_info.head(3)

The trips and vessel data are joined together.

In [ ]:
trips_combined = trips_combined.join(
    vessel_info, left_on="Vessel", right_on="VesselName", how="left", coalesce=True
)

trips_combined.head(3)

Joining in the weather data is a little harder since the weather data has a temporal resolution of one-hour whereas the trips can leave at any arbitrary time, making a naive join on timestamps not work. Here, we round the the departure time of the trips to the nearest hour and then join on the weather data.

In [ ]:
weather = weather.select(
    pl.col(
        "time",
        "weather_code",
        "temperature_2m",
        "precipitation",
        "cloud_cover",
        "wind_speed_10m",
        "wind_direction_10m",
        "wind_gusts_10m",
        "terminal_name",
    )
)

trips_combined = (
    trips_combined.with_columns(pl.col("Date").dt.round("1h").alias("time"))
    .join(
        weather.rename(lambda col_name: f"departing_{col_name}"),
        how="left",
        left_on=["Departing", "time"],
        right_on=["departing_terminal_name", "departing_time"],
        coalesce=True,
    )
    .join(
        weather.rename(lambda col_name: f"arriving_{col_name}"),
        how="left",
        left_on=["Arriving", "time"],
        right_on=["arriving_terminal_name", "arriving_time"],
        coalesce=True,
    )
    .select(pl.exclude("time"))
)

trips_combined.head(3)

Let's take a look at how many `null`s there are in the data.

In [ ]:
trips_combined.null_count()

We want the drop the `null`s except for `YearRebuilt` since we expect to have weather data available. We can expect `YearRebuilt` in the case of vessels that have not yet been rebuilt.

In [ ]:
import polars.selectors as cs

trips_combined = trips_combined.drop_nulls(subset=cs.exclude("YearRebuilt"))

trips_combined.null_count()

Now we enumerate all our numerical and categorical features. This will be needed later in setting up preprocessing for our model. For the categorical features we take a count of how often each category occurs to make sure our model isn't dependant on categories that are uncommon in the data.

In [ ]:
numeric_features = [
    "SpeedInKnots",
    "EngineCount",
    "Horsepower",
    "MaxPassengerCount",
    "YearBuilt",
    "YearRebuilt",
    "departing_temperature_2m",
    "departing_cloud_cover",
    "departing_wind_speed_10m",
    "departing_wind_direction_10m",
    "departing_wind_gusts_10m",
    "arriving_temperature_2m",
    "arriving_cloud_cover",
    "arriving_wind_speed_10m",
    "arriving_wind_direction_10m",
    "arriving_wind_gusts_10m",
]

categorical_features = [
    "Vessel",
    "Weekday",
    "Hour",
    "Departing",
    "Arriving",
    "ClassName",
    "PropulsionInfo",
    "departing_weather_code",
    "arriving_weather_code",
]

for cf in categorical_features:
    print(
        f"feature: '{cf}', count:",
        trips_combined.group_by(cf).agg(pl.len()).sort("len"),
    )

We can see that some of the weather codes don't occur as frequently as would be ideal and decide to keep note of which codes these are.

In [ ]:
low_count_weather_codes = set(
    [
        *trips_combined.group_by("departing_weather_code")
        .agg(pl.len())
        .sort("len")
        .filter(pl.col("len") < 300)["departing_weather_code"]
        .to_list(),
        *trips_combined.group_by("arriving_weather_code")
        .agg(pl.len())
        .sort("len")
        .filter(pl.col("len") < 300)["arriving_weather_code"]
        .to_list(),
    ]
)

low_count_weather_codes

These less common weather codes are binned into their own category.

In [ ]:
def recode_weather_codes(code):
    return "other" if code in low_count_weather_codes else str(code)


trips_combined = trips_combined.with_columns(
    pl.col("departing_weather_code").map_elements(
        recode_weather_codes, return_dtype=pl.String
    ),
    pl.col("arriving_weather_code").map_elements(
        recode_weather_codes, return_dtype=pl.String
    ),
)

Finally, some unnecessary columns are dropped and the `Date` column is turned into a proper date.

In [ ]:
trips_combined = trips_combined.select(
    cs.exclude("ScheduledDepart", "ActualDepart")
).with_columns(pl.col("Date").dt.date())

trips_combined

## Task 3 - Model Training

### 🔄 Task

Define a `scikit-learn` pipeline that

- Transform the data for the model to ingest
- Trains a gradient boosted machine model to predict the logged departure delay

### 🧑‍💻 Code

Here a column transformer is defined to preprocess our data. Numerical and categorical columns are treated differently with the former being passed through as-is and the latter being one-hot encoded.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

column_transformer = ColumnTransformer(
    [
        # this just passes the variables through as-is
        ("numeric_features", "passthrough", numeric_features),
        # this one-hot encodes the variables
        ("categorical_features", OneHotEncoder(), categorical_features),
    ]
)

We next define our actual model, a [gradient boosted machine](https://scikit-learn.org/stable/modules/ensemble.html#histogram-based-gradient-boosting) akin to [LightGBM](https://github.com/Microsoft/LightGBM).

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

regressor = HistGradientBoostingRegressor(verbose=2, random_state=2)

Our model requires non-sparse data, so a custom transformer is defined to take a sparse dataset and turn it into a dense one. See [toarray](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.toarray.html#scipy.sparse.csr_matrix.toarray).

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator

class DenseTransformer(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None, **params):
        return self

    def transform(self, X, y=None, **params):
        return X.toarray()

Now the transformers and model are combined into a single pipeline.

In [ ]:
from sklearn.pipeline import Pipeline

model = Pipeline(
    [
        ("column-transformer", column_transformer),
        ("densify", DenseTransformer()),
        ("regressor", regressor),
    ]
)

We set aside more recent for [model monitoring](../04-model-monitoring/monitoring_dashboard.qmd).

In [ ]:
import datetime

train_test_data = trips_combined.filter(
    pl.col("Date") < (datetime.date.today() - datetime.timedelta(weeks=4))
)

monitoring_data = trips_combined.filter(
    pl.col("Date") >= (datetime.date.today() - datetime.timedelta(weeks=4))
)

monitoring_data.write_database(
    table_name=f"{username}_monitoring_data",
    connection=db_uri,
    engine="adbc",
    if_table_exists="replace",
)

In [ ]:
Now the remaining data is split into a training set and testing set for evaluating the model.

In [ ]:
from sklearn.model_selection import train_test_split

X = train_test_data.drop("LogDelay", "Date")
y = train_test_data["LogDelay"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

print(f"Number of rows of training data: {X_train.shape[0]}")
print(f"Number of rows testing data:  {X_test.shape[0]}")

X_test.with_columns(y_test).write_database(
    table_name=f"{username}_test_data",
    connection=db_uri,
    engine="adbc",
    if_table_exists="replace",
)

In [ ]:
Finally, our model is trained. This also trains the encodings for our one-hot encoder.

In [ ]:
%%time
model.fit(X_train.to_pandas(), y_train)

The model is evaluated.

In [ ]:
model.score(X_test, y_test)

## Task 4 - Model Deployment

### 🔄 Task

- Deploy the model using `vetiver` and `pins` onto Posit Connect
- Deploy an API around the model onto Posit

### 🧑‍💻 Code

Our model is wrapped up in a [`VetiverModel` object for serving it](https://rstudio.github.io/vetiver-python/stable/reference/VetiverModel.html#vetiver.VetiverModel).

In [ ]:
from vetiver import VetiverModel

v = VetiverModel(
    model, model_name=f"{username}/ferry_delay", prototype_data=X.to_pandas()
)

Our model is now deployed to Connect, making it available to use elsewhere.

In [ ]:
import pins
import vetiver

model_board = pins.board_connect(allow_pickle_read=True)
vetiver.vetiver_pin_write(model_board, model=v)

In order to more easily use our model, we need to define an API that loads and serves it. This API will also be deployed to Connect.

In [ ]:
import shutil

Path("api").mkdir(parents=True, exist_ok=True)

# vetiver.write_app(model_board, f"{username}/ferry_delay", file="api/app.py")
shutil.copy2("requirements.txt", "api/requirements.txt")

In [ ]:
%%time
api_guid = os.getenv("API_GUID")
if api_guid is None:
    !rsconnect deploy fastapi --entrypoint app:api --title "{username}/ferry_delay_vetiver" api/
else:
    !rsconnect deploy fastapi --app-id "{api_guid}" --entrypoint app:api --title "{username}/ferry_delay_vetiver" api/

## Task 5 - Model Card

### 🔄 Task

- Use a model card to describe various metrics for how the model performs
- Deploy the card to Connect

### 🧑‍💻 Code

In [ ]:
# vetiver.templates.model_card()